In [2]:
import numpy as np
import pickle
import json
import os
import shutil
import glob

In [3]:
def get_test_info (agent_name):
    source_file = f"../agent_code/agent_{agent_name}/logs/params_test.json"
    with open(source_file, 'r') as file:
        params = json.load(file)
    
    #print(params['test'])
    model_name = params['test']['MODEL_NAME'] 

    print(f"Collecting test analysis data from {agent_name}_{model_name}.")

    return model_name

In [4]:
def get_training_info (agent_name):
    source_file = f"../agent_code/agent_{agent_name}/logs/params_train.json"
    with open(source_file, 'r') as file:
        params = json.load(file)
    
    model_name = params['training']['MODEL_NAME']
    n_rounds   = params['training']['TRAINING_ROUNDS']
    
    if 'Q_SAVE_INTERVAL' in params['Q-update']:
        Q_interval = params['Q-update']['Q_SAVE_INTERVAL']
    else:
        Q_interval = 1
    if 'Q_UPDATE_INTERVAL' in params['Q-update']:
        Q_interval *= params['Q-update']['Q_UPDATE_INTERVAL']

    print(f"Collecting training analysis data from {agent_name}_{model_name}.")
    print(f"Trained {n_rounds} rounds with a Q-model saved every {Q_interval} rounds.")

    return model_name, n_rounds, Q_interval

In [5]:
def make_analysis_directory (agent_name, model_name):
    directory = f"./{agent_name}/{model_name}/"
    os.makedirs(directory)

In [6]:
def copy_params (agent_name, model_name, mode = "train"):
    source_file      = f"../agent_code/agent_{agent_name}/logs/params_{mode}.json"
    destination_file = f"./{agent_name}/{model_name}/params_{mode}.json"

    shutil.copy(source_file, destination_file)

In [7]:
def copy_model (agent_name, model_name):
    source_file        = f"../agent_code/agent_{agent_name}/models/model_{agent_name}_{model_name}.npy"
    destination_folder = f"./{agent_name}/{model_name}/"

    shutil.copy(source_file, destination_folder)

In [8]:
def copy_sa_counter (agent_name, model_name):
    source_file      = f"../agent_code/agent_{agent_name}/models/sa_counter_{agent_name}_{model_name}.npy"
    destination_folder = f"./{agent_name}/{model_name}/"

    shutil.copy(source_file, destination_folder)

In [9]:
def copy_log (agent_name, model_name, mode = "train"):
    source_file      = f"../agent_code/agent_{agent_name}/logs/agent_{agent_name}.log"
    destination_file = f"./{agent_name}/{model_name}/log_{mode}.log"

    shutil.copy(source_file, destination_file)

In [10]:
def move_results (agent_name, model_name, mode = "train"):
    source_file        = sorted(glob.glob("../results/*.json"))[-1]   # latest json log file
    destination_folder = f"./{agent_name}/{model_name}/"
    destination_file   = f"{destination_folder}results_{mode}.json"

    if destination_file not in glob.glob(destination_folder):
        shutil.move(source_file, destination_file)
    else:
        print(f"There's already a 'results_{mode}.json' file in '{destination_folder}'.")

In [11]:
def collect_Q_data (agent_name, model_name, number_of_rounds, Q_interval):
    source_folder = f"../agent_code/agent_{agent_name}/logs/Q_data/"
    Q_file_name   = lambda x: f"{source_folder}Q{x}.npy"
    Q_file        = lambda x: np.load(Q_file_name(x))
  
    number_of_Q_files = number_of_rounds // Q_interval
    allQ       = np.zeros((number_of_Q_files, *Q_file(Q_interval).shape))
    round = Q_interval
    for i in range(number_of_Q_files):
        round    = (i+1) * Q_interval
        print(f"Loading 'Q{round}'", end="\r")
        allQ[i]  = Q_file(round)
    print('\n')

    analysis_folder = f"../analysis/{agent_name}/{model_name}/"
    allQ_file_name  = f"{analysis_folder}Qtrain.npy"
    # Check to prevent accidental overwrites
    if os.path.exists(allQ_file_name):
        print(f"Do you want to overwrite the Qtrain.py file of agent_{agent_name}_{model_name}? (y/n)")
        answer = ""
        while not (answer == "y" or answer == "n"):  
            answer = input()
        if answer == "n":
            print("Didn't overwrite.")
            return
        else:
            print(f"Overwriting '{allQ_file_name}'.")
    np.save(allQ_file_name, allQ)

    if allQ_file_name in glob.glob(f"{analysis_folder}*.npy"):
        print(f"Removing Q-files in '{source_folder}'.")
        for file in glob.glob(f"{source_folder}Q*.npy"):
            os.remove(file)
    print("Done.")

In [12]:
def copy_analysis_template (agent_name, model_name, mode = "train"):
    source_file        = f"./templates/analysis_{mode}_{agent_name}.ipynb"
    destination_folder = f"./{agent_name}/{model_name}/"

    shutil.copy(source_file, destination_folder)

In [13]:
def collect_analysis_data (agent_name, mode = "train"):
    if mode == "train":
        model_name, n_rounds, Q_interval = get_training_info(agent_name)
        make_analysis_directory(agent_name, model_name)
        copy_model             (agent_name, model_name)
        copy_sa_counter        (agent_name, model_name)
        collect_Q_data         (agent_name, model_name, n_rounds, Q_interval)
    elif mode == "test":
        model_name = get_test_info(agent_name)
    copy_params            (agent_name, model_name, mode)
    copy_log               (agent_name, model_name, mode)
    move_results           (agent_name, model_name, mode)
    copy_analysis_template (agent_name, model_name, mode)

In [31]:
collect_analysis_data("h6", "train")

Trained 500 rounds with a Q-model saved every 5 rounds.
Loading 'Q500'

Removing Q-files in '../agent_code/agent_h6/logs/Q_data/'.
Done.


In [24]:
collect_analysis_data("h6", "test")

In [22]:
copy_model("h6", "hand-crafted")

In [14]:
ideal_q_model = np.array(
    [[[0, 0, 0, 0, 1, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0]],

     [[0, 0, 0, 0, 1, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0],
      [0, 0, 0, 1, 0, 0]],

     [[0, 0, 0, 0, 1, 0],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1],
      [0, 0, 0, 0, 0, 1]]]
)

In [16]:
ideal_q_model.shape

(3, 15, 6)

In [17]:
ideal_q_model = np.swapaxes(ideal_q_model, 0, 1)
ideal_q_model.shape

(15, 3, 6)

In [18]:
directory = "../agent_code/agent_h6/models/"
filename  = "model_h6_hand-crafted.npy"

In [20]:
np.save(directory+filename, ideal_q_model)